In [26]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

In [27]:
# 读取数据
rent_train = pd.read_csv('../../dataset/RentForecast/train_data.csv')
rent_test = pd.read_csv('../../dataset/RentForecast/test_a.csv')

target_train = rent_train.iloc[:, -1]
# target_test = rent_test.pop('tradeMoney')

In [34]:
rent_train.dtypes

ID                      int64
area                  float64
rentType               object
houseType              object
houseFloor             object
totalFloor              int64
houseToward            object
houseDecoration        object
communityName          object
city                   object
region                 object
plate                  object
buildYear              object
saleSecHouseNum         int64
subwayStationNum        int64
busStationNum           int64
interSchoolNum          int64
schoolNum               int64
privateSchoolNum        int64
hospitalNum             int64
drugStoreNum            int64
gymNum                  int64
bankNum                 int64
shopNum                 int64
parkNum                 int64
mallNum                 int64
superMarketNum          int64
totalTradeMoney         int64
totalTradeArea        float64
tradeMeanPrice        float64
tradeSecNum             int64
totalNewTradeMoney      int64
totalNewTradeArea       int64
tradeNewMe

In [28]:
# 相关系数法特征选择
from sklearn.feature_selection import SelectKBest

print(rent_train.shape)

sk = SelectKBest(k=150)
new_train = sk.fit_transform(rent_train, target_train)
print(new_train.shape)

(41440, 51)


ValueError: could not convert string to float: '未知方式'

In [17]:
list_a = [1, 2, 3, 4, 5, 10]
num = list_a.pop(5)

In [18]:
num

10